<a href="https://colab.research.google.com/github/davidgvad/Image-Identifier-AI/blob/main/HW1_ngramLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1: N-Gram Language Models (50 Points)

Authors: Kabir Ahuja, Sofia Serrano

Thanks to Melissa Mitchell and Khushi Khandelwal for feedback and Kavel Rao for designing the autograder.

In this project, you will implement and experiment with N-gram language models. N-gram language models are simplest versions of a language model, which make a simplifying assumption that the probability of predicting a word in a sentence only depends on the past $N$ words in the sentence. In this project you will learn:

- How to train word-level unigram and N-gram LMs on text data
- Evaluating quality of an LM by computing perplexity
- Sampling text from an N-gram LM
- Implement Laplace Smoothing
- Implement Interpolation for N-Gram Language Models

We will be working with Shakespeare Plays from Andrej Karpathy's [blog post on Recurrent neural networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/). We also recommend going through chapter 3 of [Jurafsky and Martin](https://web.stanford.edu/~jurafsky/slp3/3.pdf) on N-Gram models, especially if you are not familiar with them yet.

In [64]:
%%bash

# get any other necessary files for this project

if [ ! -e "data-needed.txt" ]; then
  if [ ! -e "data_path_to_download_url.py" ]; then
    wget https://raw.githubusercontent.com/serrano-s/NLPassignments-students/refs/heads/main/data_path_to_download_url.py
  else
    echo "data_path_to_download_url.py script already downloaded to runtime"
  fi

  wget https://raw.githubusercontent.com/serrano-s/NLPassignments-students/refs/heads/main/assignments/NgramLanguageModels/data-needed.txt

  # download all data files needed for the student-release version of this project (i.e., no hidden test files)
  DATA_NEEDED_FILE="data-needed.txt"
  closing_slash="/"
  while IFS= read -r line; do
    line="$(echo -e "${line}" | sed -e 's/^[[:space:]]*//' -e 's/[[:space:]]*$//')";
    dirs_to_check="${line%${closing_slash}*}"
    mkdir -p $dirs_to_check
    download_url=$(python data_path_to_download_url.py "$line")
    echo $download_url;
    wget "$download_url" -O "$line"
  done < "$DATA_NEEDED_FILE"
else
  echo "data-needed.txt (and presumably therefore all necessary data files) already downloaded to runtime"
fi

if [ ! -e "other-setup-needed.sh" ]; then
  wget https://raw.githubusercontent.com/serrano-s/NLPassignments-students/refs/heads/main/assignments/NgramLanguageModels/other-setup-needed.sh
  bash other-setup-needed.sh
  rm data_path_to_download_url.py
else
  echo "other-setup-needed.sh (and presumably therefore all other necessary files) already downloaded to runtime"
fi

data-needed.txt (and presumably therefore all necessary data files) already downloaded to runtime
other-setup-needed.sh (and presumably therefore all other necessary files) already downloaded to runtime


In [65]:
# Load necessary packages
import os
from typing import List, Dict
import random
import numpy as np

# Setup nltk
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [66]:
# Set data directory. Important: DO NOT CHANGE THIS. MY AUTOGRADER WILL FAIL ON YOUR SUBMISSION OTHERWISE
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, "data")

In [67]:
# Helper functions for sample test cases

def evaluate_test_case(input, output, expected_output, output_str = "Output", atol=1e-4) -> Dict:

    if input is not None:
        print("Input:\n", input)
    print(f"{output_str}:\n", output)
    print(f"Expected {output_str}:\n", expected_output)

    match = (
        output == expected_output
        if type(output) == str
        else np.allclose(output, expected_output, atol=atol)
    )

    if match:
        print("Test case passed! :)")

    else:
        print("Test case failed! :(")

    print("\n" + "="*50 + "\n")


def evaluate_list_test_case(input: List, output: List, expected_output: List) -> Dict:

    print("Input:\n", input)
    print("Output:\n", output)
    print("Expected output:\n", expected_output)
    if output == expected_output:
        print("Test case passed! :)")

    else:
        print("Test case failed! :(")

    print("\n" + "="*50 + "\n")

## Part 1: Word-level unigram language models (11 points)

We will start by considering word-level language models i.e. language models where the smallest unit (or a unigram) that can be predicted by the model is a word. In part 1, we will implement unigram language models, which constitutes the simplest variant of N-gram models -- simply learn the distribution of each unigram (here a word) in the corpus. Recall that for a text sequence with unigrams $w_1, w_2, \cdots, w_n$, unigram language models, the probability of the sequence is given as:

$$P(w_1, w_2, \cdots, w_n) =P(w_1)P(w_2)\cdots P(w_n)$$

where $P(w_i)$ is simply the frequency of the word $w_i$ in the training corpus.

For this part we will work with the Shakespeare dataset. Let's start by loading the datasets

In [68]:
with open(f"{data_dir}/shakespeare/shakespeare_train.txt") as f:
    train_data = f.read().split("\n")

with open(f"{data_dir}/shakespeare/shakespeare_dev.txt") as f:
    dev_data = f.read().split("\n")

Below we print first 10 sentences from the training data.

In [69]:
print("\n".join(train_data[:10]))

First Citizen : Before we proceed any further , hear me speak .
All : Speak , speak .
First Citizen : You are all resolved rather to die than to famish ?
All : Resolved .
resolved .
First Citizen : First , you know Caius Marcius is chief enemy to the people .
All : We know't , we know't .
First Citizen : Let us kill him , and we 'll have corn at our own price .
Is't a verdict ?
All : No more talking o n't ; let it be done : away , away !


### Exercise 1.0 Text Processing

Before start training our models, let's perform some basic preprocessing. For this exercise, we only want to put an \<eos\> tag at the end of each sentence in the training and dev sets. Implement `add_eos` function below that does that.

In [70]:
def add_eos(data: List[str]) -> List[str]:
    """
    Adds an <eos> token to the end of each line in the data.

    Inputs:
    - data: a list of strings where each string is a line of text

    Returns:
    - a list of strings where each string is a line of text with <eos> token appended
    """
    output = []
    for line in data:
        # remove trailing whitespace
        line = line.rstrip()
        # If the line is empty just return "<eos>" to avoid " <eos>"
        if line == "":
            output.append("<eos>")
        else:
            output.append(line + " <eos>")

    return output


In [71]:
def test_add_eos():
    print("Running Sample Test Case 1")
    data = ["hello!", "world!"]

    evaluate_list_test_case(data, add_eos(data), ["hello! <eos>", "world! <eos>"])

    print("Running Sample Test Case 2")
    data = [
        "Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ." ]

    expected_output = [
        "Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice . <eos>",
        "At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs . <eos>",
        "The world was so recent that many things lacked names, and in order to indicate them it was necessary to point . <eos>"]
    evaluate_list_test_case(data, add_eos(data), expected_output)

test_add_eos()

Running Sample Test Case 1
Input:
 ['hello!', 'world!']
Output:
 ['hello! <eos>', 'world! <eos>']
Expected output:
 ['hello! <eos>', 'world! <eos>']
Test case passed! :)


Running Sample Test Case 2
Input:
 ['Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .', 'At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .', 'The world was so recent that many things lacked names, and in order to indicate them it was necessary to point .']
Output:
 ['Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice . <eos>', 'At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished

In [72]:
train_data_processed = add_eos(train_data)
dev_data_processed = add_eos(dev_data)

### Exercise 1.1: Training (word-level) Unigram Language Model (2 Points)

Training a unigram model simply corresponds to calculating frequencies of each word in the corpus, i.e.

$$p(w_i) = \frac{C(w_i)}{n}$$

where $C(w_i)$ is the count of word $w_i$ in the training data and $n$ is the total number of words in the training dataset.

Implement the `train_word_unigram` function below

In [73]:
def train_word_unigram(train_data: List[str]) -> Dict[str, float]:

    """
    Trains a word-level unigram language model.

    Inputs:
        - train_data: List[str], list of sentences in training data

    Outputs:
        - Dict[str, float], a dictionary mapping words to their unigram probabilities

    """

    unigram_probs = {}
    word_counts: Dict[str, int] = {}
    total_tokens = 0

    # count tokens
    for sent in train_data:
        tokens = sent.strip().split() #keeps <eos> as one token
        for tok in tokens:
            word_counts[tok] = word_counts.get(tok, 0) + 1
            total_tokens += 1

    # convert counts to probabilities
    unigram_probs: Dict[str, float] = {}
    if total_tokens == 0:
        return unigram_probs

    for tok, c in word_counts.items():
        unigram_probs[tok] = c / total_tokens
    return unigram_probs

In [74]:
unigram_probs = train_word_unigram(train_data_processed)

In [75]:
# Sample test cases
def test_train_word_unigram(unigram_probs):

    print("Running Sample Test Case 1: Check if the number of unique words is correct")
    evaluate_test_case(None, len(unigram_probs), 12610, output_str="Number of unique words")

    print("Running Sample Test Case 2: Check if the probability of word \"thou\" is correct")
    evaluate_test_case("thou", unigram_probs["thou"], 0.004559649641510829)

    print("Running Sample Test Case 3: Check if the probability of word \"love\" is correct")
    evaluate_test_case("love", unigram_probs["love"], 0.0015984182127282134)

    print("Running Sample Test Case 4: Check if the probability of word \"Richard\" is correct")
    evaluate_test_case("Richard", unigram_probs["Richard"], 0.0005035479340675586)

    print("Running Sample Test Case 5: Check if the probability of word \"pytorch\" is correct")
    evaluate_test_case("pytorch", unigram_probs.get("pytorch", 0), 0)

    print("Running Sample Test Case 6: Check if the probability of word \"richard\" is correct")
    evaluate_test_case("richard", unigram_probs.get("richard", 0), 0)


test_train_word_unigram(unigram_probs)

Running Sample Test Case 1: Check if the number of unique words is correct
Number of unique words:
 12610
Expected Number of unique words:
 12610
Test case passed! :)


Running Sample Test Case 2: Check if the probability of word "thou" is correct
Input:
 thou
Output:
 0.004559649641510829
Expected Output:
 0.004559649641510829
Test case passed! :)


Running Sample Test Case 3: Check if the probability of word "love" is correct
Input:
 love
Output:
 0.0015984182127282134
Expected Output:
 0.0015984182127282134
Test case passed! :)


Running Sample Test Case 4: Check if the probability of word "Richard" is correct
Input:
 Richard
Output:
 0.0005035479340675586
Expected Output:
 0.0005035479340675586
Test case passed! :)


Running Sample Test Case 5: Check if the probability of word "pytorch" is correct
Input:
 pytorch
Output:
 0
Expected Output:
 0
Test case passed! :)


Running Sample Test Case 6: Check if the probability of word "richard" is correct
Input:
 richard
Output:
 0
Expected

### Exercise 1.2: Evaluating (word-level) Unigram Language Models using Perplexity (2 Points)

Now that we have trained our first (albeit very basic) language model, our next job is to evaluate how good of a job it does in modeling the training text as well as generalizing on the unseen text. The most commonly used metric for evaluating the quality of a language model is Perplexity. Recall from the lecture, perplexity of a language model on a test dataset measures the (inverse) probability assigned by the language model to the test dataset normalized by the number of words (or tokens). Lower the perplexity the higher probability the model assigns to the text in the test dataset and hence better quality.

$$\text{perplexity}(W) = P(w_1w_2\cdots w_n)^{\frac{-1}{n}} = \sqrt[n]{\frac{1}{P(w_1w_2\cdots w_n)}}$$

where $W$ is a test set with $n$ words $w_1w_2\cdots w_n$

It is useful to do perplexity calculation in log space to avoid numerical issues

$$\text{perplexity}(W) = \exp\bigl(-\frac{\log{P(w_1w_2\cdots w_n)}}{n}\bigr)$$

When we have multiple sentences in the corpus and we assume sentences to be independent, we can write:

$$\text{perplexity}(W) = \exp\bigl(-\frac{ \sum_{S \in W}  \log{P(s_1s_2\cdots s_{n_s})}}{n}\bigr)$$

where $S$ is a sentence in the corpus $W$ with words $s_1 s_2 \cdots s_{n_s}$ and $n_s$ is the number of words in $S$.

Note that assuming sentences to be independent is something that is not actually true in practice. However, since N-gram language models are already limited in their context, it is not the greatest loss to remove dependencies between sentences. In the future homeworks we will be dropping this assumption as we build more powerful models.


In [76]:
def eval_ppl_word_unigram(eval_data: List[str], unigram_probs: Dict[str, float]) -> float:

    """
    Evaluates the perplexity of a word-level unigram language model on the dev set.

    Inputs:
        - dev_data: List[str], list of sentences in the evaluation data
        - unigram_probs: Dict[str, float], a dictionary mapping words to their unigram probabilities

    Outputs:
        - float, the perplexity of the unigram language model on the evaluation set

    Note 1: It is useful to do the calculations in log space and convert the final answer to original space to
    avoid numerical issues .
    Note 2: Assign 0 probability to words that are not in the unigram_probs dictionary, since those words were not seen during training.
    """
    import math
    perplexity = 0
    total_log_prob = 0.0
    total_tokens = 0
    for sent in eval_data:
        tokens = sent.strip().split()
        for tok in tokens:
            p = unigram_probs.get(tok, 0.0) # 0 for unseen words
            if p == 0.0:
                return float("inf") # log(0) = -inf, perplexity = inf
            total_log_prob += math.log(p) # natural log
            total_tokens += 1

    if total_tokens == 0:
        return 1.0  #empty eval set

    perplexity = math.exp(-total_log_prob / total_tokens)

    return perplexity

First let's compute the perplexity of training data

In [77]:
train_ppl = eval_ppl_word_unigram(train_data_processed, unigram_probs)
print(train_ppl)

575.1110328373742


You should expect a perplexity around 575 on the training data. Now let's evaluate on dev data.

In [78]:
dev_ppl = eval_ppl_word_unigram(dev_data_processed, unigram_probs)
print(dev_ppl)

inf


You should see a RuntimeWarning: divide by zero and a perplexity of infinity in this case. Why did this happen? Because we have some words in the dev dataset, which were never seen during training. The unigram model assigns a zero probability to those, which result in an infinite perplexity (inverse probability).

### Exercise 1.3: Handling Unknown Words (2 Points)

How do we deal with such situations? An easy solution is to introduce an unknown word token, e.g. \<unk\>. Before training, we replace the words which occur less than a threshold number of times with an \<unk\> token. E.g., if a word occurs less than 3 times in the dataset, we replace it with the \<unk\> token. At test time, when evaluating the perplexity if we see a word, which was unseen during training i.e. has a unigram frequency of 0, we assign that word the probability of \<unk\> token.

Implement the `replace_rare_words_with_unks` function below which replaces words which occur less than `unk_thresh` number of times by \<unk\> token. Also reimplement `eval_ppl_word_unigram` to handle unseen words. You might find the [`Counter` class from the `collections` module useful](https://docs.python.org/3/library/collections.html#counter-objects).

In [79]:
def replace_rare_words_with_unks(train_data: List[str], unk_thresh: int) -> List[str]:
    """
    Replaces words that occur less than unk_thresh times in the training data with "<unk>" token.

    Inputs:
        - train_data: List[str], list of sentences in the training data
        - unk_thresh: int, the threshold on the number of occurrences of a word to be considered known (less than considered <unk>)

    Outputs:
        - List[str], the training text with rare words replaced by "<unk>" token
    """
    from collections import Counter

    train_data_unked = []
    # recount token frequencies
    all_tokens = []
    for sent in train_data:
        all_tokens.extend(sent.strip().split())
    counts = Counter(all_tokens)
    # Replace rare tokens
    for sent in train_data:
        tokens = sent.strip().split()
        new_tokens = []
        for tok in tokens:
            if tok == "<eos>":
                new_tokens.append(tok)
            elif counts[tok] < unk_thresh:
                new_tokens.append("<unk>")
            else:
                new_tokens.append(tok)
        train_data_unked.append(" ".join(new_tokens))

    return train_data_unked

In [80]:
# Sample test cases
def test_train_word_unigram_wth_unks():

    print("# Testing for Threshold 3\n\n")
    train_data_wth_unks = replace_rare_words_with_unks(train_data_processed, 3)
    unigram_probs_wth_unks = train_word_unigram(train_data_wth_unks)
    print("Sample Test Case 1: Check if the number of unique words is correct")
    evaluate_test_case(None, len(unigram_probs_wth_unks), 4620, output_str="Number of unique words")

    print("Sample Test Case 2: Check if the probability of word \"<unk>\" is correct")
    evaluate_test_case("<unk>", unigram_probs_wth_unks["<unk>"], 0.045185342597383396)

    print("# Testing for threshold 5\n\n")
    train_data_wth_unks = replace_rare_words_with_unks(train_data_processed, 5)
    unigram_probs_wth_unks = train_word_unigram(train_data_wth_unks)
    print("Sample Test Case 3: Check if the number of unique words is correct")
    evaluate_test_case(None, len(unigram_probs_wth_unks), 3088, output_str="Number of unique words")

    print("Sample Test Case 4: Check if the probability of word \"<unk>\" is correct")
    evaluate_test_case("<unk>", unigram_probs_wth_unks["<unk>"], 0.06910155961268387)

test_train_word_unigram_wth_unks()

# Testing for Threshold 3


Sample Test Case 1: Check if the number of unique words is correct
Number of unique words:
 4620
Expected Number of unique words:
 4620
Test case passed! :)


Sample Test Case 2: Check if the probability of word "<unk>" is correct
Input:
 <unk>
Output:
 0.045185342597383396
Expected Output:
 0.045185342597383396
Test case passed! :)


# Testing for threshold 5


Sample Test Case 3: Check if the number of unique words is correct
Number of unique words:
 3088
Expected Number of unique words:
 3088
Test case passed! :)


Sample Test Case 4: Check if the probability of word "<unk>" is correct
Input:
 <unk>
Output:
 0.06910155961268387
Expected Output:
 0.06910155961268387
Test case passed! :)




In [81]:
# We will remove words that occur less than 3 times
train_data_wth_unks = replace_rare_words_with_unks(train_data_processed, 3)
unigram_probs_wth_unks = train_word_unigram(train_data_wth_unks)

We recommend you to check the unigram probabilities before and after replacing rare words with \<unk\> token to verify your implementation.

In [82]:
def eval_ppl_word_unigram_with_unks(eval_data: List[str], unigram_probs_wth_unks: Dict[str, float]) -> float:

    """
    Evaluates the perplexity of a word-level unigram language model on the dev set. For unseen words, uses the <unk> token probability.

    Inputs:
        - eval_data: string, List of sentences in eval data
        - unigram_probs_wth_unks: Dict[str, float], a dictionary mapping words to their unigram probabilities, including <unk> token

    Outputs:
        - float, the perplexity of the unigram language model on the evaluation set

    """
    import math

    perplexity = 0
    total_log_prob = 0.0
    total_tokens = 0
    unk_prob = unigram_probs_wth_unks.get("<unk>", 0.0)

    for sent in eval_data:
        for tok in sent.strip().split():
            p = unigram_probs_wth_unks.get(tok, unk_prob)
            if p == 0.0:
                return float("inf")

            total_log_prob += math.log(p)
            total_tokens += 1

    if total_tokens == 0:
        return 1.0

    perplexity=math.exp(-total_log_prob / total_tokens)

    return perplexity

Let's compute the train and dev perplexity now

In [83]:
train_ppl = eval_ppl_word_unigram_with_unks(train_data_wth_unks, unigram_probs_wth_unks)
print(f"Train Perplexity: {train_ppl}")
dev_ppl = eval_ppl_word_unigram_with_unks(dev_data_processed, unigram_probs_wth_unks)
print(f"Dev Perplexity: {dev_ppl}")

Train Perplexity: 384.0815655120125
Dev Perplexity: 282.0959070802668


You should observe a train perplexity around 384 and dev perplexity around 282.

### Exercise 1.4: Sampling from Unigram Language Model (2 Points)

Now that we have trained and evaluated our unigram LM, we are ready to generate some text from it. To sample text from an N-gram language model given prefix words $w_1, w_2, \cdots, w_n$, we sequentially sample next tokens from the N-gram probability distribution given the previous words, i.e.,

$$w_{n+1} \sim P(w_{n+1} | w_{1}, \cdots, w_{n} )$$

For a unigram language model, since $P(w_1, \dots, w_n) = P(w_1)\cdots P(w_n)$ i.e. all words all distributed independently and the next token is sampled independent of previous tokens, the above equation simplifies to:

$$w_{n+1} \sim P(w_{n+1})$$

You will now implement the function `sample_from_word_unigram` below. You might find the [Numpy's random module (np.random)](https://numpy.org/doc/stable/reference/random/index.html) useful.

In [84]:
def sample_from_word_unigram(unigram_probs: Dict[str, float], max_words: int, prefix: str = "") -> str:

    """
    Samples sequence of words from a unigram language model.
    Terminate sampling when either max_words is reached or when <eos> token is sampled.

    Inputs:
        - unigram_probs: Dict[str, float], a dictionary mapping words to their unigram probabilities
        - n_words: int, the number of words to sample
        - prefix: str, a prefix to start the sampling from. Can have multiple words separated by spaces.

    Outputs:
        - str: sampled text i.e. string of sampled words separated by spaces along with the prefix

    # Note: Please use np.random.choice to sample from the unigram_probs dictionary.
    """
    sampled_string = ""

    # Convert dict
    words = list(unigram_probs.keys())
    probs = np.array([unigram_probs[w] for w in words], dtype=float)
    # normalize because of floating error
    probs = probs / probs.sum()
    # Start with prefix
    out_tokens = []
    if prefix.strip() != "":
        out_tokens.extend(prefix.strip().split())
    for _ in range(max_words):
        next_word = np.random.choice(words, p=probs)
        if next_word == "<eos>":
            break
        out_tokens.append(next_word)
    sampled_string = " ".join(out_tokens)
    return sampled_string

In [85]:
# Sample test cases
def test_sample_from_word_unigram():

    np.random.seed(0)
    sampled_string = sample_from_word_unigram(unigram_probs_wth_unks, 10, "The king")
    print("Running Test Case 1: Check if the sampled string starts with the prefix")
    evaluate_test_case(None, sampled_string.startswith("The king"), True, output_str="Sampled string starts with the prefix")

    print("Running Test Case 2: Check if the sampled string has either 10 generated words or ends with <eos> token")
    print(f"Generated string: {sampled_string}")
    print(f"Number of generated words: {len(sampled_string.split()) - 2}")
    print(f"Does the generated string end with <eos> token: {'<eos>' in sampled_string}")
    if len(sampled_string.split()) - 2 == 10 or (
        len(sampled_string.split()) - 2 < 10 and "<eos>" in sampled_string
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")
    print("\n" + "="*50 + "\n")

    print("Running Test Case 3: Check if the probability of generating <unk> token is correct")

    sampled_strings = [
        sample_from_word_unigram(unigram_probs_wth_unks, 1, "")
        for _ in range(1000)
    ]
    sampled_string = " ".join(sampled_strings)
    num_unks = sampled_string.count("<unk>")
    unk_gen_prob = num_unks / len(sampled_string.split())
    evaluate_test_case(None, unk_gen_prob, unigram_probs_wth_unks["<unk>"], output_str="Probability of generating <unk> token", atol=1e-2)

test_sample_from_word_unigram()

Running Test Case 1: Check if the sampled string starts with the prefix
Sampled string starts with the prefix:
 True
Expected Sampled string starts with the prefix:
 True
Test case passed! :)


Running Test Case 2: Check if the sampled string has either 10 generated words or ends with <eos> token
Generated string: The king If thee For no that body of weeping drunk it
Number of generated words: 10
Does the generated string end with <eos> token: False
Test passed! :)


Running Test Case 3: Check if the probability of generating <unk> token is correct
Probability of generating <unk> token:
 0.053180396246089674
Expected Probability of generating <unk> token:
 0.045185342597383396
Test case passed! :)




Note that due to the randomness in sampling, it is difficult to automatically test this function. However, you can use the technique we use in sample test case 3 by repeatedly sampling from the unigram model and checking the frequency of different words in the generated text and checking if they are close to the unigram probabilities.

Let's sample some text from the unigram model

In [86]:
for _ in range(10):
    sampled_string = sample_from_word_unigram(unigram_probs_wth_unks, 20)
    print(sampled_string)

where , know this : if in <unk> do How <unk> by To OF brow God in
Shepherd . I , KING aspiring Than our <unk> <unk> . country . the yet high : that thus Do
I GAUNT <unk> : I <unk> willing <unk> the She thee here thou while : : 's , , None
deed . First redemption you
From , <unk> grandsire To his ! less . VI voice Of , thou Hold his a me So dost
, letter : forces will The is joys <unk> feast JULIET knee what , I of are that a
of <unk> his : queen Call The right immediately , love ANGELO wearing For . go :
John and To you unjustly said and return , done to Not , ! <unk> he our art : A
? <unk> thy realm the foot , A Shall
that royal , : now of thy LADY the be I do PARIS Rome be <unk> KING . sun Henry


You should see that the generated that doesn't make a whole lot of sense, which is natural since we are using a unigram model that doesn't account for the context at all, and essentially samples the most common tokens in its generations. We will now move to build language models that do not have this problem, i.e., they take into account the context (albeit to different degrees) for modeling the distribution of words (or tokens) in the text.

### Write-Up Question 1: Effect of \<unk\> tokens on perplexity and generation quality (2 Points)

What effect do you think the \<unk\> tokens will have on the perplexity of the model? Try out different thresholds for replacing rare words with \<unk\> token and report the perplexity on the training and dev set. What do you observe? Does the perplexity decrease with increasing threshold? Do you think that improves generation quality? (Your answer to this question should go in your separate write-up PDF.)

**What to submit:**

- A Table with the perplexity of the unigram model on the training and dev set for different thresholds of replacing rare words with \<unk\> token. You can choose the thresholds as 1, 3, 5,  7, 9, 10.

- Example generations at each threshold.

- 3-4 lines maximum discussing the trends you observe in the perplexity and generation quality with increasing threshold.

In [24]:

# YOUR CODE HERE


### Write-Up Question 2: Alterative to Random Sampling (1 Point)

An alternate algorithm to generate text from a language model is greedy decoding, i.e., where we generate the most likely token at each step of decoding, i.e.

$$ w_{k+1} = \texttt{argmax}_{w} P(w | w_{1}, \cdots, w_{k}) $$

Can you explain why or why not that will be a good idea for unigram language models? Explain in no more in 3 lines. (Your answer to this question should go in your separate write-up PDF.)

## Part 2: N(>1)-Gram Word-Level Language Models (12 Points)

We will now implement much more sophisticated language models, which make use of the surrounding text to model the distribution of text. Recall from the lectures for an $N$-gram language model with $n > 1$, the distribution of a sequence of tokens $w_1, w_2, \cdots, w_n$ is given as:

$$P(w_1, w_2, \cdots, w_n) = \prod_{k=1}^{n}P(w_k \mid w_{k-N-1}, \cdots, w_{k-1})$$

E.g., for a bigram model i.e. $N = 2$, the expression becomes:

$$P(w_1, w_2, \cdots, w_n) = \prod_{k=1}^{n}P(w_k \mid w_{k-1})$$

i.e. the distribution of a token depends solely on the tokens preceding it.

### Write-Up Question 3 (2 Points)

Can you show why for an N-gram language model the following expression holds?

$$P(w_1, w_2, \cdots, w_n) = \prod_{k=1}^{n}P(w_k \mid w_{k-N-1}, \cdots, w_{k-1})$$

Lay down the assumption that is required to derive this expression and show all the steps in your derivation. (Your answer to this question should go in your separate write-up PDF.)


### Exercise 2.1: Text Processing

A careful reader must have noted that the above expression: $P(w_1, w_2, \cdots, w_n) = \prod_{k=1}^{n}P(w_k \mid w_{k-1})$ has a problem. On the right hand side, when $k = 1$, this will result in the term $P(w_1 | w_0)$, but there is no $w_0$ in the sequence. Similarly, for a trigram language model we will have terms, $P(w_1 | w_{-1}, w_{0})$ and $P(w_2 | w_0, w_1)$ with conditionals on words that are not part of the sequence. To handle this issue, we add $N-1$ start of sequence tokens, e.g. \<sos\> to the beginning of the sequence.

Implement `process_text_for_Ngram` function below that adds $N-1$ \<sos\> tokens to beginning of each sentence in the dataset.

In [87]:
def process_text_for_Ngram(sents: List[str], N: int = 2) -> List[str]:

    """
    Adds N-1 <sos> tokens to the start of every sentence in the text.

    Inputs:
        - sents: List[str], List of sentences
        - N: int, the N in N-gram

    Outputs:
        - List[str], the processed text
    """
    processed_sents = []
    sos_prefix = " ".join(["<sos>"] * (N - 1))  # "" if N=1
    for sent in sents:
        sent = sent.strip()
        if N <= 1:
            processed_sents.append(sent)
        else:
            if sent == "":
                processed_sents.append(sos_prefix)
            else:
                processed_sents.append(sos_prefix + " " + sent)

    return processed_sents

In [89]:
# Sample test cases

def test_process_text_for_Ngram():

    sents = ["Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
    "At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
    "The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ." ]

    print("Running Sample Test Case 1 with N=1")
    unigram_processed_text = process_text_for_Ngram(sents, 1)
    excepted_output = [
        "Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ."
    ]
    evaluate_list_test_case(sents, unigram_processed_text, excepted_output)

    print("Running Sample Test Case 2 with N=2")
    bigram_processed_text = process_text_for_Ngram(sents, 2)
    excepted_output = [
        "<sos> Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "<sos> At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "<sos> The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ."
    ]
    evaluate_list_test_case(sents, bigram_processed_text, excepted_output)

    print("Running Sample Test Case 3 with N=3")
    trigram_processed_text = process_text_for_Ngram(sents, 3)
    excepted_output = [
        "<sos> <sos> Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "<sos> <sos> At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "<sos> <sos> The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ."
    ]
    evaluate_list_test_case(sents, trigram_processed_text, excepted_output)

    print("Running Sample Test Case 4 with N=4")
    fourgram_processed_text = process_text_for_Ngram(sents, 4)
    excepted_output = [
        "<sos> <sos> <sos> Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .",
        "<sos> <sos> <sos> At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .",
        "<sos> <sos> <sos> The world was so recent that many things lacked names, and in order to indicate them it was necessary to point ."
    ]
    evaluate_list_test_case(sents, fourgram_processed_text, excepted_output)

test_process_text_for_Ngram()

Running Sample Test Case 1 with N=1
Input:
 ['Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .', 'At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .', 'The world was so recent that many things lacked names, and in order to indicate them it was necessary to point .']
Output:
 ['Many years later , as he faced the firing squad , Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice .', 'At that time Macondo was a village of twenty adobe houses, built on the bank of a river of clear water that ran along a bed of polished stones, which were white and enormous, like prehistoric eggs .', 'The world was so recent that many things lacked names, and in order to indicate them it was necessary 

### Exercise 2.2: Implementing N-gram language models (10 Points)

The heart of implementing an N-gram language model is to estimate the conditional distributions $P(w_n \mid w_{n-N-1}, \cdots, w_{n-1})$. Recall from the lectures that the conditional distributions can be estimated as:

$$P(w_n \mid w_{n-N-1}, \cdots, w_{n-1}) = \frac{C(w_{n-N-1} \cdots w_{n-1} w_{n})}{\sum_{w \in V}{C(w_{n-N-1} \cdots w_{n-1} w)}} = \frac{C(w_{n-N-1} \cdots w_{n-1} w_{n})}{{C(w_{n-N-1} \cdots w_{n-1})}}$$

where $C(w_{n-N-1} \cdots w_{n-1} w)$ is the number of times the token sequence $w_{n-N-1} \cdots w_{n-1} w$ appears in the corpus, and $V$ is the vocabulary of the N-gram model.

You will now implement an N-gram language model from scratch. Note that a full implementation involves a `fit` function, which computes the N-gram counts $C(w_{n-N-1} \cdots w_{n-1} w)$ needed to compute the conditional distributions; `eval_perplexity` function, which evaluates the perplexity of the language model on a test set; and a `sample_text` function which generates the text from the LM. Implement the class `WordNGramLM` below with these functions.This time we leave all the implementation details for you to decide. We just provide a boiler plate code, with the expected input output for the three functions in the class. You might need to implement additional functions for your implementation of the three functions.

**Note 1**: If implementing a general NGram Model from scratch seems too daunting, we recommend starting implementing just the Bigram LM and checking if you are able to pass BigramLM Test cases. From there you can work on generalizing your solution.

**Note 2**: Efficiency of your code will be important here, especially for `eval_perplexity` and `sample_text` functions. A naive implementation of these functions will result in a time complexity of $\pmb{O}(nV)$ for `eval_perplexity` and $\pmb{O}(TV)$ for `sample_text`, where $n$ is the number of words in evaluation dataset, T is the length of sampled text, and $V$ is the size of vocabulary. By caching certain quantities during training, you should be able to write implementations with time complexities $\pmb{O}(n)$ and either $\pmb{O}(T + V)$ or $\pmb{O}(T)$ for the two functions. We will give only half credit for the naive implementation.

**Note 3**: We will provide sample test cases only to test `eval_perplexity` and `sample_text` functions and not for the `fit` function. Both of the former functions rely on the latter to be implemented correctly, hence use correctness of `eval_perplexity` and `sample_text` to check the correctness of your `fit` function.

In [100]:
from typing import List, Dict, Tuple
from collections import defaultdict
import numpy as np
import math
class WordNGramLM:

    def __init__(self, N: int):
        self.N = N
        self.vocab = set()

        # Counts
        self.context_counts = defaultdict(int) # context count
        self.next_counts = defaultdict(lambda: defaultdict(int))  # context - word -  count

        # Cached for fast sampling
        self._sample_cache = {}

    def fit(self, train_data: List[str]):

        """
        Trains an N-gram language model.

        Inputs:
            - train_data: str, sentences in the training data

        """
        # First Pass count all words
        word_counts = defaultdict(int)
        for sent in train_data:
            # FIX: Manually append <eos> to raw data
            sent_with_eos = sent.strip() + " <eos>"
            for tok in sent_with_eos.split():
                word_counts[tok] += 1

        # Define the vocabulary freq>=3
        self.vocab = set()
        self.vocab.add("<unk>")
        self.vocab.add("<eos>")
        if self.N > 1:
            self.vocab.add("<sos>")

        for w, c in word_counts.items():
            if c >= 3:
                self.vocab.add(w)
        # Reset vounts
        self.context_counts = defaultdict(int)
        self.next_counts = defaultdict(lambda: defaultdict(int))
        k = self.N - 1

        # Second pass, train
        for sent in train_data:
            sent_with_eos = sent.strip() + " <eos>"
            raw_tokens = sent_with_eos.split()
            # Map rare to <unk>
            tokens = []
            for tok in raw_tokens:
                if tok in self.vocab:
                    tokens.append(tok)
                else:
                    tokens.append("<unk>")
            # Add <sos> padding
            tokens = self._ensure_sos(tokens)
            if self.N == 1:
                ctx = tuple()
                for w in tokens:
                    self.context_counts[ctx] += 1
                    self.next_counts[ctx][w] += 1
            else:
                for i in range(k, len(tokens)):
                    ctx = tuple(tokens[i-k:i])
                    w = tokens[i]
                    self.context_counts[ctx] += 1
                    self.next_counts[ctx][w] += 1

        # Cache sampling distributions
        self._sample_cache = {}
        for ctx, w_counts in self.next_counts.items():
            words = np.array(list(w_counts.keys()), dtype=object)
            counts = np.array([w_counts[w] for w in words], dtype=float)
            probs = counts / counts.sum()
            self._sample_cache[ctx] = (words, probs)

    def eval_perplexity(self, eval_data: List[str]) -> float:

        """
        Evaluates the perplexity of the N-gram language model on the eval set.

        Input:
            - eval_data: List[str], the evaluation text

        Output:
            - float, the perplexity of the model on the evaluation set

        Note : For words that are not in the vocabulary, replace them with the <unk> token.
        Note : Don't count the <sos> tokens in your number of total tokens in order to match expected perplexities.

        """
        total_log_prob = 0.0
        total_tokens = 0
        k = self.N - 1
        for sent in eval_data:
            # Manually append <eos> to eval data
            sent_with_eos = sent.strip() + " <eos>"
            raw_tokens = sent_with_eos.split()
            # Add sos padding
            raw_tokens = self._ensure_sos(raw_tokens)
            # Map OOV to <unk>
            tokens = []
            for t in raw_tokens:
                if t == "<sos>":
                    tokens.append(t)
                else:
                    tokens.append(self._map_oov_to_unk(t))
            if self.N == 1:
                ctx = tuple()
                for w in tokens:
                    num = self.next_counts[ctx].get(w, 0)
                    den = self.context_counts.get(ctx, 0)
                    if num == 0 or den == 0:
                        return float("inf")
                    total_log_prob += math.log(num) - math.log(den)
                    if w != "<sos>":
                        total_tokens += 1
            else:
                for i in range(k, len(tokens)):
                    ctx = tuple(tokens[i-k:i])
                    w = tokens[i]
                    den = self.context_counts.get(ctx, 0)
                    num = self.next_counts[ctx].get(w, 0)
                    if num == 0 or den == 0:
                        return float("inf")
                    total_log_prob += math.log(num) - math.log(den)
                    if w != "<sos>":
                        total_tokens += 1

        if total_tokens == 0:
            return 1.0

        return math.exp(-total_log_prob / total_tokens)

    def sample_text(self, prefix: str = "<sos>", max_words: int = 100) -> float:

        """
        Samples text from the N-gram language model.
        Terminate sampling when either max_words is reached or when <eos> token is sampled.
        Inputs:
            - prefix: str, the prefix to start the sampling from. Can also be multiple words separated by spaces.
            - max_words: int, the maximum number of words to sample

        Outputs:
            - str, the sampled text

        Note: Please use np.random.choice for sampling next words
        """
        if prefix.strip():
            prefix_tokens = prefix.strip().split()
        else:
            prefix_tokens = []

        prefix_tokens = self._ensure_sos(prefix_tokens)

        # map OOV in prefix to <unk> except for <sos>
        prefix_tokens = [t if t == "<sos>" else self._map_oov_to_unk(t) for t in prefix_tokens]
        out = list(prefix_tokens)
        k = self.N - 1

        for _ in range(max_words):
            if self.N == 1:
                ctx = tuple()
            else:
                ctx = tuple(out[-k:])

            if ctx not in self._sample_cache:
                # If a context was never seen in training, we cant sample from it so stop
                break
            words, probs = self._sample_cache[ctx]
            nxt = np.random.choice(words, p=probs)
            out.append(nxt)
            if nxt == "<eos>":
                break
        return " ".join(out)

    # Extra utility functions that you think will be useful can go below

    def _ensure_sos(self, tokens: List[str]) -> List[str]:
        k = self.N - 1
        if k <= 0:
            return tokens
        lead = 0
        while lead < len(tokens) and tokens[lead] == "<sos>":
            lead += 1
        tokens_wo_extra = tokens[lead:]
        return (["<sos>"] * k) + tokens_wo_extra

    def _map_oov_to_unk(self, tok: str) -> str:
        return tok if tok in self.vocab else "<unk>"


In [28]:
# <NO_AUTOGRADE>

As a sanity check, check if your code returns same perplexities when N=1 as your earlier unigram model perplexities

In [101]:
unigram_lm = WordNGramLM(1)
unigram_lm.fit(train_data)

train_ppl = unigram_lm.eval_perplexity(train_data)
dev_ppl = unigram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Unigram model (Class Implementation): {train_ppl}")
print(f"Dev Perplexity for Unigram model (Class Implementation): {dev_ppl}")

train_ppl_old = eval_ppl_word_unigram_with_unks(train_data_wth_unks, unigram_probs_wth_unks)
dev_ppl_old = eval_ppl_word_unigram_with_unks(dev_data_processed, unigram_probs_wth_unks)

print(f"Train Perplexity for Unigram model (Original Implementation): {train_ppl_old}")
print(f"Dev Perplexity for Unigram model (Original Implementation): {dev_ppl_old}")

if np.allclose(train_ppl, train_ppl_old, atol = 1e-4) and np.allclose(dev_ppl, dev_ppl_old, atol = 1e-4):
    print("Unigram model perplexities match! :)")

else:
    print("Unigram model perplexities do not match! :(")

# assert np.allclose(train_ppl, train_ppl_old, atol = 1e-4)
# assert np.allclose(dev_ppl, dev_ppl_old, atol = 1e-4)
# print("Unigram model perplexities match!")


Train Perplexity for Unigram model (Class Implementation): 384.0815655120125
Dev Perplexity for Unigram model (Class Implementation): 282.0959070802668
Train Perplexity for Unigram model (Original Implementation): 384.0815655120125
Dev Perplexity for Unigram model (Original Implementation): 282.0959070802668
Unigram model perplexities match! :)


Test implementation of `eval_perplexity` for bigram model

In [102]:
# Test implementation of `eval_perplexity` for bigram model
bigram_lm = WordNGramLM(2)
bigram_lm.fit(train_data)
train_ppl = bigram_lm.eval_perplexity(train_data)
print(f"Train Perplexity: {train_ppl}")
dev_ppl = bigram_lm.eval_perplexity(dev_data)
print(f"Dev Perplexity: {dev_ppl}")

Train Perplexity: 42.128332262041745
Dev Perplexity: inf


You should see a train perplexity of roughly 42 and dev perplexity infinite, we will soon see how to deal with that.

Test implementation of `sample_text` for bigram model

In [103]:
def test_sample_text_ngram():

    print("Testing for Trigram model")

    random.seed(42)
    np.random.seed(42)
    trigram_lm = WordNGramLM(3)
    trigram_lm.fit(train_data)
    sampled_text = trigram_lm.sample_text("<sos> <sos>", max_words=50)

    print("Test Case 1: Check if the sampled text starts with <sos> <sos>")
    evaluate_test_case(None, sampled_text.startswith("<sos> <sos>"), True, output_str="Sampled text starts with <sos> <sos>")

    print("Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>")
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 2}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 2 == 50 or (
        len(sampled_text.split()) - 2 < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 3: Check if the probability of generating II is greater than III when prefix is KING RICHARD")
    sampled_texts = [
        trigram_lm.sample_text("KING RICHARD", max_words=1) for _ in range(10000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("KING RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)
    num_richard_3s = [
        text.split("KING RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)

    gen_prob_richard_2 = num_richard_2s / len(sampled_texts)
    gen_prob_richard_3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_richard_2}")
    print(f"Probability of generating Richard III: {gen_prob_richard_3}")
    if gen_prob_richard_2 < gen_prob_richard_3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 4: Check if the probability of generating II given KING RICHARD are close to the expected values")
    evaluate_test_case("King Richard II", gen_prob_richard_2, 0.4051, output_str="Probability of generating Richard II", atol=1e-2)

    print("Test Case 5: Check if the probability of generating III given KING RICHARD are close to the expected values")
    evaluate_test_case("King Richard III", gen_prob_richard_3, 0.5949, output_str="Probability of generating Richard III", atol=1e-2)

    print("Testing for 4-gram model")
    random.seed(42)
    np.random.seed(42)
    fourgram_lm = WordNGramLM(4)
    fourgram_lm.fit(train_data)
    # sampled_text = fourgram_lm.sample_text("<sos> <sos>", max_words=50)

    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos>", max_words=50)

    print("Test Case 6: Check if the sampled text starts with <sos> <sos> <sos>")
    evaluate_test_case(None, sampled_text.startswith("<sos> <sos> <sos>"), True, output_str="Sampled text starts with <sos> <sos> <sos>")

    print("Test Case 7: Check if the number of generated words is either 50 or less than 50 and ends with <eos>")
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 3}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 3 == 50 or (
        len(sampled_text.split()) - 3 < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")

    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 8: Check if the probability of generating II is greater than III when prefix is <sos> KING RICHARD")
    sampled_texts = [
        fourgram_lm.sample_text("<sos> <sos> KING RICHARD", max_words=1) for _ in range(10000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("<sos> <sos> KING RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)

    num_richard_3s = [
        text.split("<sos> <sos> KING RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)

    gen_prob_rich2 = num_richard_2s / len(sampled_texts)
    gen_prob_rich3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_rich2}")
    print(f"Probability of generating Richard III: {gen_prob_rich3}")
    if gen_prob_rich2 < gen_prob_rich3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 9: Check if the probability of generating II given <sos> KING RICHARD are close to the expected values")
    evaluate_test_case("<sos> King Richard II", gen_prob_rich2, 0.4044, output_str="Probability of generating Richard II", atol=1e-2)

    print("Test Case 10: Check if the probability of generating III given <sos> KING RICHARD are close to the expected values")
    evaluate_test_case("<sos> King Richard III", gen_prob_rich3, 0.5956, output_str="Probability of generating Richard III", atol=1e-2)

test_sample_text_ngram()

Testing for Trigram model
Test Case 1: Check if the sampled text starts with <sos> <sos>
Sampled text starts with <sos> <sos>:
 True
Expected Sampled text starts with <sos> <sos>:
 True
Test case passed! :)


Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>
Generated text: <sos> <sos> Please your honour , state and seat is up on high ; Whilst you have been i ' the part I had a Harry , Harry , <unk> to you both ! <eos>
Number of generated words: 33
Does the generated text end with <eos>: True
Test passed! :)


Test Case 3: Check if the probability of generating II is greater than III when prefix is KING RICHARD
Probability of generating Richard II: 0.4052
Probability of generating Richard III: 0.5948
Test passed! :)


Test Case 4: Check if the probability of generating II given KING RICHARD are close to the expected values
Input:
 King Richard II
Probability of generating Richard II:
 0.4052
Expected Probability of generating Richard I

In [104]:
def test_sample_text_bigram_model():
    bigram_lm = WordNGramLM(2)
    bigram_lm.fit(train_data)

    random.seed(42)
    np.random.seed(42)
    sampled_text = bigram_lm.sample_text("<sos>", max_words=50)

    print("Test Case 1: Check if the sampled text starts with <sos>")
    evaluate_test_case(None, sampled_text.startswith("<sos>"), True, output_str="Sampled text starts with <sos>")

    print("Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>")
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 1}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 1 == 50 or (
        len(sampled_text.split()) < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print("Test Case 3: Check if the probability of generating II is greater than III when prefix is RICHARD")
    sampled_texts = [
        bigram_lm.sample_text("RICHARD", max_words=1) for _ in range(10000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)
    num_richard_3s = [
        text.split("RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)
    gen_prob_richard_2 = num_richard_2s / len(sampled_texts)
    gen_prob_richard_3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_richard_2}")
    print(f"Probability of generating Richard III: {gen_prob_richard_3}")
    if gen_prob_richard_2 < gen_prob_richard_3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "="*50 + "\n")

    print(
        "Test Case 4: Check if the probability of generating II given RICHARD  are close to the expected values"
    )
    evaluate_test_case("Richard II", gen_prob_richard_2, 0.35251798561151076, output_str="Probability of generating Richard II", atol=1e-2)

    print(
        "Test Case 5: Check if the probability of generating III given RICHARD are close to the expected values"
    )
    evaluate_test_case("Richard III", gen_prob_richard_3, 0.49640287769784175, output_str="Probability of generating Richard III", atol=1e-2)

test_sample_text_bigram_model()

Test Case 1: Check if the sampled text starts with <sos>
Sampled text starts with <sos>:
 True
Expected Sampled text starts with <sos>:
 True
Test case passed! :)


Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>
Generated text: <sos> Please but great and the <unk> ? <eos>
Number of generated words: 8
Does the generated text end with <eos>: True
Test passed! :)


Test Case 3: Check if the probability of generating II is greater than III when prefix is RICHARD
Probability of generating Richard II: 0.35
Probability of generating Richard III: 0.5044
Test passed! :)


Test Case 4: Check if the probability of generating II given RICHARD  are close to the expected values
Input:
 Richard II
Probability of generating Richard II:
 0.35
Expected Probability of generating Richard II:
 0.35251798561151076
Test case passed! :)


Test Case 5: Check if the probability of generating III given RICHARD are close to the expected values
Input:
 Richard 

In [105]:
for _ in range(20):
    sampled_text = bigram_lm.sample_text("<sos> KING", max_words=50)
    print(sampled_text)

<sos> KING HENRY VI KING RICHARD : That when for the sun . <eos>
<sos> KING EDWARD IV : You bless you disturb devotion shows much corn gratis . <eos>
<sos> KING RICHARD III : Why , while a <unk> ; My wretchedness doth not in harvest of dignity and another day . <eos>
<sos> KING RICHARD II : yet ? <eos>
<sos> KING RICHARD III : 'T is frank 'd A fellow : Not having in the other : <unk> tenderness and crows , That <unk> down I would lay all , I could say this night Whilst that 's son thither : WARWICK : Who , sir , And a person .
<sos> KING RICHARD III : I will never would here ? <eos>
<sos> KING HENRY VI : this noble kinsman ! <eos>
<sos> KING LEWIS XI : what we 'll swear That thou liest ' <eos>
<sos> KING HENRY BOLINGBROKE : Do me of Gaunt . <eos>
<sos> KING RICHARD III : Amen , my soul and revenge , or any , wretched self , And not speak to them still , I shall pay that sends to time craves to me way : nay . <eos>
<sos> KING EDWARD : Why , stay to Saint <unk> but I am no more ; for us 

The generations should look much better now, way more coherent compared to the unigram model! At least on the surface level it seems to capture the style of Shakespeare's writing.

Test implementation of `eval_perplexity` for trigram, 4-gram, and 5-gram models

In [106]:
trigram_lm = WordNGramLM(3)
trigram_lm.fit(train_data)

train_ppl = trigram_lm.eval_perplexity(train_data)
dev_ppl = trigram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Trigram model: {train_ppl}")
print(f"Dev Perplexity for Trigram model: {dev_ppl}")

print("\n\n")

fourgram_lm = WordNGramLM(4)
fourgram_lm.fit(train_data)
train_ppl = fourgram_lm.eval_perplexity(train_data)
dev_ppl = fourgram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for 4-gram model: {train_ppl}")
print(f"Dev Perplexity for 4-gram model: {dev_ppl}")

print("\n\n")

fivegram_lm = WordNGramLM(5)
fivegram_lm.fit(train_data)
train_ppl = fivegram_lm.eval_perplexity(train_data)
dev_ppl = fivegram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for 5-gram model: {train_ppl}")
print(f"Dev Perplexity for 5-gram model: {dev_ppl}")

Train Perplexity for Trigram model: 5.939635966646537
Dev Perplexity for Trigram model: inf



Train Perplexity for 4-gram model: 2.0705701088539454
Dev Perplexity for 4-gram model: inf



Train Perplexity for 5-gram model: 1.595255728780846
Dev Perplexity for 5-gram model: inf


You should see train perplexities approximately 5.9, 2.1, and 1.6 for trigram, 4-gram, and 5-gram LMs. For dev data the dev perplexity should be infinity for all the three.

Test implementation of `sample_text` for trigram and 4-gram LMs.

In [107]:
np.random.seed(42)
print("Generations from Trigram model")
for _ in range(20):
    sampled_text = trigram_lm.sample_text("<sos> <sos> KING", max_words=50)
    print(sampled_text)

Generations from Trigram model
<sos> <sos> KING RICHARD II : I am known to go . <eos>
<sos> <sos> KING EDWARD IV : Is it not amiss ; I will to sanctuary . <eos>
<sos> <sos> KING RICHARD II : Where <unk> , behold 's : there she lost a brace . <eos>
<sos> <sos> KING EDWARD IV : Brave warriors , Clifford , take heed , for Lancaster ! <eos>
<sos> <sos> KING HENRY VI : My lord , the fool was that of common soldiers slain . <eos>
<sos> <sos> KING EDWARD IV : Go thither ; For this will teach thee how to curse mine enemies ! <eos>
<sos> <sos> KING RICHARD III : Say no more hear from you The apprehension of the news abroad ? <eos>
<sos> <sos> KING RICHARD II : We have made fault I ' shall poison , go in your city is <unk> 'd out an act of parliament be repeal 'd , Nay , now at my injustice . <eos>
<sos> <sos> KING RICHARD III : O Marcius , Attend upon Cominius to these arms , and did scorn an humble tear ; And therefore , not for your country 's foes , to <unk> and <unk> 'd : say it is . <eos>


In [108]:
np.random.seed(42)
random.seed(42)
print("Generations from 4-gram model")
for _ in range(20):
    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

Generations from 4-gram model
<sos> <sos> <sos> KING RICHARD II : I am content . <eos>
<sos> <sos> <sos> KING RICHARD II : Twice for one step I 'll groan , the way being short , And piece the way out with a heavy heart . <eos>
<sos> <sos> <sos> KING HENRY VI : <unk> men , much <unk> with care , Here sits a king more woful than you are hurt by me . <eos>
<sos> <sos> <sos> KING RICHARD III : As I remember , this should be the first word of thy speech : For this , being <unk> in sap and blood , your flesh and blood has not offended the king ; and signify to him That thus I have resign 'd my charge to
<sos> <sos> <sos> KING RICHARD III : Now , messenger , what letters or what news From France ? <eos>
<sos> <sos> <sos> KING HENRY VI : Ay , but mildly . <eos>
<sos> <sos> <sos> KING RICHARD III : How chance the prophet could not at that time , <unk> the loving kiss I give the fruit . <eos>
<sos> <sos> <sos> KING RICHARD II : A king of beasts , indeed ; and therefore came I hither . <eos>
<sos

In [109]:
np.random.seed(42)
random.seed(42)
print("Generations from 5-gram model")
for _ in range(20):
    sampled_text = fivegram_lm.sample_text("<sos> <sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

Generations from 5-gram model
<sos> <sos> <sos> <sos> KING RICHARD II : I am in this , Your wife , your son , these senators , the nobles ; And you will rather show our general <unk> How you can frown than spend a fawn upon 'em , For the inheritance of their loves and safeguard Of what that
<sos> <sos> <sos> <sos> KING LEWIS XI : And mine with hers , and thine , and Margaret 's . <eos>
<sos> <sos> <sos> <sos> KING EDWARD IV : He 's sudden , if a thing comes in his head . <eos>
<sos> <sos> <sos> <sos> KING RICHARD III : Well , sir ; my mistress is the sweetest lady -- Lord , Lord ! <eos>
<sos> <sos> <sos> <sos> KING RICHARD III : March on , march on , since we are up in arms ; If not to fight with foreign enemies , Yet to beat down these rebels here at home . <eos>
<sos> <sos> <sos> <sos> KING RICHARD II : Twice for one step I 'll groan , the way being short , And piece the way out with a bloody axe . <eos>
<sos> <sos> <sos> <sos> KING RICHARD III : Well , let that pass . <eos>
<sos> <s

In [38]:
# </NO_AUTOGRADE>

You should see that the generation quality is now so much better. However, if you look closely (this is specially true for 4-gram and 5-gram models) that some of the sentences are directly lifted from the training data. Which makes sense, as we will increase the order of the N-gram LM, so does we increase its capacity and hence more the ability to memorize its training data.

## Part 3: Smoothing and Interpolation (27 Points)

The issue with using N-gram language models is that any finite training corpus is bound to miss some N-grams that appear in the test set. The models hence assign zero probability to such N-grams, leading to probability of the entire test set to be zero and hence infinite perplexity values that we observed in the previous exercise.

The standard way to deal with zero-probability N-gram tokens is to use smoothing algorithms. Smoothing algorithms shave off a bit of probability mass from some more frequent events and give it to unseen events. Smoothing algorithms for N-gram language models is a well studied area of research with numerous algorithms. For this exercise, we will focus on Laplace Smoothing and Interpolation.

**Hint:** Try to use the inheritance from `WordNGramLM` to simplify your implementations for this part. You can use `super().method(arguments)` to call a method from the super class.

### Exercise 3.1 Laplace and Add-Lambda Smoothing (10 Points)

Perhaps the simplest smoothing algorithm that exists is Laplace smoothing. It merely adds one to count of each N-gram, so that there is no zero-probability N-gram in the test data. For a bigram model, the expression for Laplace-smoothened distribution is given by:

$$P_{\text{Laplace}}(w_n \mid w_{n-1}) = \frac{C(w_{n-1}w_{n}) + 1}{\sum_{w \in V} (C(w_n w) + 1) } = \frac{C(w_{n-1}w_{n}) + 1}{ C(w_{n-1}) + |V| }$$

We can similarly write expressions for other N-gram models.

Laplace smoothing is also called "Add-one" smoothing. A generalization of Laplace Smoothing is "Add-k" smoothing with  with $\lambda < 1$, where we move a bit less of the probability mass from seen to unseen N-grams. The expression for Add-k smoothened distribution for bigram LM is given by:

$$P_{\text{Add-k}}(w_n \mid w_{n -1}) = \frac{C(w_{n-1}w_{n}) + \lambda}{\sum_{w \in V} (C(w_n w) + \lambda) } = \frac{C(w_{n-1}w_{n}) + \lambda}{ C(w_{n-1}) + \lambda |V| }$$

We will use the variable `k` to represent Lambda in the code.

For this exercise, we ask you to implement `WordNGramLMWithAddKSmoothing` class. You need to follow the same code structure as `WordNGramLM` class, with only difference being in calculation of the conditional distributions.

Note: It is no longer possible to implement an O(T + V) time complexity solution for `sample_text` function, hence we will accept O(TV) implementations here. Your `eval_perplexity` should still be as efficient as in the `WordNGramLM`


In [121]:
class WordNGramLMWithAddKSmoothing(WordNGramLM):
    """
    Remember you can use the inheritance from WordNGramLM in your implementation!
    """

    def __init__(self, N: int, k: int = 1):
        super().__init__(N)
        self.k = k
        self._pred_vocab = []
        self._V = 0

    def fit(self, train_data: List[str]):
        """
        Trains an N-gram language model with Add-k smoothing.

        Inputs:
            - train_data: str, sentences in the training data

        """
        # First pass count words to find rare ones
        word_counts = defaultdict(int)
        for sent in train_data:
            sent_processed = self._preprocess_line(sent)
            for tok in sent_processed.split():
                word_counts[tok] += 1

        # Define Vocab
        self.vocab = set()
        self.vocab.add("<unk>")
        self.vocab.add("<eos>")
        if self.N > 1:
            self.vocab.add("<sos>")

        for w, c in word_counts.items():
            if c >= 3:
                self.vocab.add(w)

        # Reset counts
        self.context_counts = defaultdict(int)
        self.next_counts = defaultdict(lambda: defaultdict(int))
        k = self.N - 1

        # Second Pass, train
        for sent in train_data:
            sent_processed = self._preprocess_line(sent)
            raw_tokens = sent_processed.split()
            tokens = []
            for tok in raw_tokens:
                if tok in self.vocab:
                    tokens.append(tok)
                else:
                    tokens.append("<unk>")

            tokens = self._ensure_sos(tokens)
            if self.N == 1:
                ctx = tuple()
                for w in tokens:
                    self.context_counts[ctx] += 1
                    self.next_counts[ctx][w] += 1
            else:
                for i in range(k, len(tokens)):
                    ctx = tuple(tokens[i-k:i])
                    w = tokens[i]
                    self.context_counts[ctx] += 1
                    self.next_counts[ctx][w] += 1

        # Pre calc prediction vocab for sampling
        if self.N > 1:
            self._pred_vocab = sorted(list(self.vocab - {"<sos>"}))
        else:
            self._pred_vocab = sorted(list(self.vocab))
        self._V = len(self._pred_vocab)

        if self._V == 0:
            raise ValueError("Vocab is empty after fit")

    def eval_perplexity(self, eval_data: List[str]) -> float:
        """
        Evaluates the perplexity of the N-gram language model with Add-k smoothing on the eval set.

        Input:
            - eval_data: List[str], the evaluation text

        Output:
            - float, the perplexity of the model on the evaluation set

        Note : For tokens that are not in the vocabulary, replace them with the <unk> token.

        """
        total_log_prob = 0.0
        total_tokens = 0
        kctx = self.N - 1
        V = self._V
        k = self.k
        for sent in eval_data:
            # hleper preprocessing here
            sent_processed = self._preprocess_line(sent)
            raw_tokens = sent_processed.split()
            raw_tokens = self._ensure_sos(raw_tokens)

            tokens = []
            for t in raw_tokens:
                if t == "<sos>":
                    tokens.append(t)
                else:
                    tokens.append(self._map_oov_to_unk(t))

            if self.N == 1:
                ctx = tuple()
                den = self.context_counts.get(ctx, 0) + k * V
                for w in tokens:
                    num = self.next_counts[ctx].get(w, 0) + k
                    total_log_prob += math.log(num) - math.log(den)
                    if w != "<sos>":
                        total_tokens += 1
            else:
                for i in range(kctx, len(tokens)):
                    ctx = tuple(tokens[i-kctx:i])
                    w = tokens[i]
                    den = self.context_counts.get(ctx, 0) + k * V
                    num = self.next_counts[ctx].get(w, 0) + k
                    total_log_prob += math.log(num) - math.log(den)
                    if w != "<sos>":
                        total_tokens += 1

        if total_tokens == 0:
            return 1.0
        return math.exp(-total_log_prob / total_tokens)

    def sample_text(
        self, prefix: str = "<sos>", max_words: int = 100,
    ) -> float:
        """
        Samples text from the N-gram language model.

        Inputs:
            - prefix: str, the prefix to start the sampling from. Can also be multiple words separated by spaces.
            - max_words: int, the maximum number of words to sample

        Outputs:
            - str, the sampled text

        Note: Please use np.random.choice for sampling next words
        """
        if prefix.strip():
            prefix_tokens = prefix.strip().split()
        else:
            prefix_tokens = []
        prefix_tokens = self._ensure_sos(prefix_tokens)
        prefix_tokens = [t if t == "<sos>" else self._map_oov_to_unk(t) for t in prefix_tokens]
        out = list(prefix_tokens)
        kctx = self.N - 1
        k = float(self.k)
        pred_vocab = self._pred_vocab
        V = self._V

        for _ in range(max_words):
            if self.N == 1:
                ctx = tuple()
            else:
                ctx = tuple(out[-kctx:])

            ctx_count = self.context_counts.get(ctx, 0)
            den = ctx_count + k * V
            counts_for_ctx = self.next_counts.get(ctx, {})

            probs = np.array([(counts_for_ctx.get(w, 0) + k) / den for w in pred_vocab], dtype=float)
            probs = probs / probs.sum()

            nxt = np.random.choice(pred_vocab, p=probs)
            out.append(nxt)
            if nxt == "<eos>":
                break

        return " ".join(out)

    # Extra utility functions that you think will be useful can go below

    def _preprocess_line(self, line: str) -> str:
            line = line.strip()
            if not line.endswith("<eos>"):
                line += " <eos>"
            return line




In [40]:
# <NO_AUTOGRADE>

Test implementation of `eval_perplexity` for bigram model with Laplace smoothing

In [122]:
# Test implementation of `eval_perplexity` for bigram model with Laplace smoothing
bigram_lm = WordNGramLMWithAddKSmoothing(2, k = 1)
bigram_lm.fit(train_data)
train_ppl = bigram_lm.eval_perplexity(train_data)
print(f"Train Perplexity: {train_ppl}")
dev_ppl = bigram_lm.eval_perplexity(dev_data)
print(f"Dev Perplexity: {dev_ppl}")

Train Perplexity: 403.44737905989643
Dev Perplexity: 390.0445447032354


You should get a train perplexity around 404 and dev perplexity around 390. Notice how the dev perplexity is not $\infty$ anymore! Though it comes at the cost of an increase in perplexity on training data, which is natural since the model now cut offs some probability mass from training N-grams and distribute it to the unseen ones.

Test implementation of `eval_perplexity` for bigram model with Add-k smoothing (k = 0.01)

In [123]:
# Test implementation of `eval_perplexity` for bigram model with Add-k smoothing
bigram_lm = WordNGramLMWithAddKSmoothing(2, k=0.01)
bigram_lm.fit(train_data)
train_ppl = bigram_lm.eval_perplexity(train_data)
print(f"Train Perplexity: {train_ppl}")
dev_ppl = bigram_lm.eval_perplexity(dev_data)
print(f"Dev Perplexity: {dev_ppl}")

Train Perplexity: 60.78322177710043
Dev Perplexity: 147.92340187733126


You should get a train perplexity around 61 and a dev perplexity of roughly 148. Notice how we do much better on train perplexity when k is smaller, since now we re-distribute much less of the mass from the training N-grams. Luckily, in this case it turns out it improves the dev perplexity too.

Test implementation of `sample_text` for bigram model with Laplace Smoothing

In [124]:
def test_sample_text_bigram_laplace_model():
    bigram_lm = WordNGramLMWithAddKSmoothing(2, k = 1)
    bigram_lm.fit(train_data_wth_unks)

    random.seed(42)
    np.random.seed(42)
    sampled_text = bigram_lm.sample_text("<sos>", max_words=50)

    print("Test Case 1: Check if the sampled text starts with <sos>")
    evaluate_test_case(
        None,
        sampled_text.startswith("<sos>"),
        True,
        output_str="Sampled text starts with <sos>",
    )

    print(
        "Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>"
    )
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 1}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 1 == 50 or (
        len(sampled_text.split()) < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 3: Check if the probability of generating II is greater than III when prefix is RICHARD"
    )
    sampled_texts = [
        bigram_lm.sample_text("RICHARD", max_words=1) for _ in range(1000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)
    num_richard_3s = [
        text.split("RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)
    gen_prob_richard_2 = num_richard_2s / len(sampled_texts)
    gen_prob_richard_3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_richard_2}")
    print(f"Probability of generating Richard III: {gen_prob_richard_3}")
    if gen_prob_richard_2 < gen_prob_richard_3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 4: Check if the probability of generating II given RICHARD  are close to the expected values"
    )
    evaluate_test_case(
        "Richard II",
        gen_prob_richard_2,
        0.014,
        output_str="Probability of generating Richard II",
        atol=1e-3,
    )

    print(
        "Test Case 5: Check if the probability of generating III given RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "Richard III",
        gen_prob_richard_3,
        0.034,
        output_str="Probability of generating Richard III",
        atol=1e-3,
    )

test_sample_text_bigram_laplace_model()

Test Case 1: Check if the sampled text starts with <sos>
Sampled text starts with <sos>:
 True
Expected Sampled text starts with <sos>:
 True
Test case passed! :)


Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>
Generated text: <sos> LADY waked quickly lords Ross Stands EDWARD succeed lovest prating Antiates whistle south all Tower Volsces chamber hire fancy bustle married RUTLAND butcher defence fools secrets act heavenly live Boy making Truly FLORIZEL vulgar what sith changes LEONTES page fellow My hap Both torment between occasion chose herein innocence Whereto
Number of generated words: 50
Does the generated text end with <eos>: False
Test passed! :)


Test Case 3: Check if the probability of generating II is greater than III when prefix is RICHARD
Probability of generating Richard II: 0.02
Probability of generating Richard III: 0.037
Test passed! :)


Test Case 4: Check if the probability of generating II given RICHARD  are clo

In [125]:
random.seed(42)
np.random.seed(42)
bigram_lm = WordNGramLMWithAddKSmoothing(2, k = 1)
bigram_lm.fit(train_data)
for _ in range(20):
    sampled_text = bigram_lm.sample_text("<sos> KING", max_words=50)
    print(sampled_text)

<sos> KING chose wall quench lose Standing Stanley Doricles summers loves praise Answer whistle south all Tower Volsces chamber hire fancy bustle married RUTLAND butcher defence fools secrets act heavenly live Boy making Truly FLORIZEL vulgar what sith changes LEONTES page fellow My hap Both torment between occasion chose herein innocence Whereto
<sos> KING whence sanctuary varlet thou livest truth Hath accomplish Citizens common do blushing songs dare bred infant Return shore GREY worldly sadly accused 'em sisterhood powerful pursues rust GREGORY danger More sunshine might consequence Even clear complete push moveables the great Montgomery prevent ride kindly rush grief hereafter faintly BLUNT Making
<sos> KING Boy motion cleft haunt tonight been elder respected arrived Gives bush Than unruly side monster swallow should Worthy thinks imposition shunn thousands coldly Master approved fair sixteen success 'not health envious antic Nor could vice commons her piece deck why weeping begg g

You should see generations which are much less like training data! <sos> KING is now followed by words other than name of king names like before. Though the generations are much less clean now.

In [126]:
random.seed(42)
np.random.seed(42)
bigram_lm = WordNGramLMWithAddKSmoothing(2, k=0.01)
bigram_lm.fit(train_data)
for _ in range(20):
    sampled_text = bigram_lm.sample_text("<sos> KING", max_words=50)
    print(sampled_text)

<sos> KING HENRY raw prime merit Great Isabella , their late milk , will show HENRY BOLINGBROKE : Hold him Although ceremonious lips . <eos>
<sos> KING HENRY VI complaint Ye harvest mew 'd isle Hence , when you shall be <unk> me against Warwick Hang All this alteration muse consequence hag helm Repair whereon south unnatural sunshine lief trembles Delphos SCROOP : I fear . <eos>
<sos> KING HENRY BOLINGBROKE : Ay comest Could you say , 't please to speak paid Hadst enjoin Had thy majesty Give ROMEO : I say it true bounty Clown cloud retire imagine relation gloves glasses fools ' <eos>
<sos> KING EDWARD IV : No quiet actions exceed seal Would I can bite waking sear me to the advantage thieves hear the town amiss Gaoler Page dogs sixteen unrest 're lineal exile Men My fortunes unnatural aunt fitter place ? <eos>
<sos> KING lie <unk> at friend . <eos>
<sos> KING RICHARD III : He was a <unk> as your country festival relation ah pomp commandment niece mar her . <eos>
<sos> KING HENRY BOLINGB

Notice how with a smaller value of $k$, the generated sentences now start to resemble more with the training data

Test implementation of `eval_perplexity` for trigram, 4-gram, and 5-gram LMs with Laplace smoothing

In [127]:
trigram_lm = WordNGramLMWithAddKSmoothing(3, k =1)
trigram_lm.fit(train_data_wth_unks)

train_ppl = trigram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = trigram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Trigram model: {train_ppl}")
print(f"Dev Perplexity for Trigram model: {dev_ppl}")

print("\n\n")

fourgram_lm = WordNGramLMWithAddKSmoothing(4, k =1)
fourgram_lm.fit(train_data_wth_unks)
train_ppl = fourgram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = fourgram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for 4-gram model: {train_ppl}")
print(f"Dev Perplexity for 4-gram model: {dev_ppl}")

print("\n\n")

fivegram_lm = WordNGramLMWithAddKSmoothing(5, k =1)
fivegram_lm.fit(train_data_wth_unks)
train_ppl = fivegram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = fivegram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for 5-gram model: {train_ppl}")
print(f"Dev Perplexity for 5-gram model: {dev_ppl}")

Train Perplexity for Trigram model: 1276.2291220834627
Dev Perplexity for Trigram model: 1829.0124278822302



Train Perplexity for 4-gram model: 1709.2873613126283
Dev Perplexity for 4-gram model: 3040.724184459413



Train Perplexity for 5-gram model: 1793.4599605426565
Dev Perplexity for 5-gram model: 3337.8334986966984


You should roughly observe the following perplexities:

|N-gram LM | Train Perplexity | Dev Perplexity|
|----------|------------------|---------------|
| Trigram   | 1276              | 1829           |
| 4-gram  | 1710              | 3041         |
| 5-gram | 1794 | 3339 |


Test implementation of `sample_text` for trigram and 4-gram LMs with Laplace Smoothing

In [128]:
def test_sample_text_ngram_laplace():
    print("Testing for Trigram model")

    random.seed(42)
    np.random.seed(42)
    trigram_lm = WordNGramLMWithAddKSmoothing(3)
    trigram_lm.fit(train_data)
    sampled_text = trigram_lm.sample_text("<sos> <sos>", max_words=50)

    print("Test Case 1: Check if the sampled text starts with <sos> <sos>")
    evaluate_test_case(
        None,
        sampled_text.startswith("<sos> <sos>"),
        True,
        output_str="Sampled text starts with <sos> <sos>",
    )

    print(
        "Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>"
    )
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 2}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 2 == 50 or (
        len(sampled_text.split()) - 2 < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 3: Check if the probability of generating II is less than III when prefix is KING RICHARD"
    )
    sampled_texts = [
        trigram_lm.sample_text("KING RICHARD", max_words=1) for _ in range(10000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("KING RICHARD")[1].strip() == "II" for text in sampled_texts
    ].count(True)
    num_richard_3s = [
        text.split("KING RICHARD")[1].strip() == "III" for text in sampled_texts
    ].count(True)

    gen_prob_richard_2 = num_richard_2s / len(sampled_texts)
    gen_prob_richard_3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_richard_2}")
    print(f"Probability of generating Richard III: {gen_prob_richard_3}")
    if gen_prob_richard_2 < gen_prob_richard_3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 4: Check if the probability of generating II given KING RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "King Richard II",
        gen_prob_richard_2,
        0.0201,
        output_str="Probability of generating Richard II",
        atol=1e-2,
    )

    print(
        "Test Case 5: Check if the probability of generating III given KING RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "King Richard III",
        gen_prob_richard_3,
        0.0306,
        output_str="Probability of generating Richard III",
        atol=1e-2,
    )

    print("Testing for 4-gram model")
    fourgram_lm = WordNGramLMWithAddKSmoothing(4)
    fourgram_lm.fit(train_data)
    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos>", max_words=50)

    print("Test Case 6: Check if the sampled text starts with <sos> <sos> <sos>")
    evaluate_test_case(
        None,
        sampled_text.startswith("<sos> <sos> <sos>"),
        True,
        output_str="Sampled text starts with <sos> <sos> <sos>",
    )

    print(
        "Test Case 7: Check if the number of generated words is either 50 or less than 50 and ends with <eos>"
    )
    print(f"Generated text: {sampled_text}")
    print(f"Number of generated words: {len(sampled_text.split()) - 3}")
    print(f"Does the generated text end with <eos>: {'<eos>' in sampled_text}")
    if len(sampled_text.split()) - 3 == 50 or (
        len(sampled_text.split()) - 3 < 50 and "<eos>" in sampled_text
    ):
        print("Test passed! :)")

    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 8: Check if the probability of generating II is less than III when prefix is <sos> KING RICHARD"
    )
    sampled_texts = [
        fourgram_lm.sample_text("<sos> <sos> KING RICHARD", max_words=1)
        for _ in range(1000)
    ]
    sampled_text = " ".join(sampled_texts)
    num_richard_2s = [
        text.split("<sos> <sos> KING RICHARD")[1].strip() == "II"
        for text in sampled_texts
    ].count(True)

    num_richard_3s = [
        text.split("<sos> <sos> KING RICHARD")[1].strip() == "III"
        for text in sampled_texts
    ].count(True)

    gen_prob_rich2 = num_richard_2s / len(sampled_texts)
    gen_prob_rich3 = num_richard_3s / len(sampled_texts)

    print(f"Probability of generating Richard II: {gen_prob_rich2}")
    print(f"Probability of generating Richard III: {gen_prob_rich3}")
    if gen_prob_rich2 < gen_prob_rich3:
        print("Test passed! :)")
    else:
        print("Test failed! :(")

    print("\n" + "=" * 50 + "\n")

    print(
        "Test Case 9: Check if the probability of generating II given <sos> KING RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "<sos> King Richard II",
        gen_prob_rich2,
        0.0174,
        output_str="Probability of generating Richard II",
        atol=1e-2,
    )

    print(
        "Test Case 10: Check if the probability of generating III given <sos> KING RICHARD are close to the expected values"
    )
    evaluate_test_case(
        "<sos> King Richard III",
        gen_prob_rich3,
        0.0295,
        output_str="Probability of generating Richard III",
        atol=1e-2,
    )
test_sample_text_ngram_laplace()

Testing for Trigram model
Test Case 1: Check if the sampled text starts with <sos> <sos>
Sampled text starts with <sos> <sos>:
 True
Expected Sampled text starts with <sos> <sos>:
 True
Test case passed! :)


Test Case 2: Check if the number of generated words is either 50 or less than 50 and ends with <eos>
Generated text: <sos> <sos> LADY waked quickly lordship Stands Stands ELBOW summers lovest pray Antigonus whistle sour alliance Welshmen Where changed ho far butchers marry Remember caitiff defy for secure act heavier live Cold manage Troy Faith wait what side changes LEWIS pays felt OVERDONE groan Bound torments between obedient chosen herein innocents While
Number of generated words: 50
Does the generated text end with <eos>: False
Test passed! :)


Test Case 3: Check if the probability of generating II is less than III when prefix is KING RICHARD
Probability of generating Richard II: 0.0206
Probability of generating Richard III: 0.0301
Test passed! :)


Test Case 4: Check if the

In [129]:
np.random.seed(42)
random.seed(42)
print("Generations from Trigram model")
trigram_lm = WordNGramLMWithAddKSmoothing(3, k = 0.01)
trigram_lm.fit(train_data)
for _ in range(20):
    sampled_text = trigram_lm.sample_text("<sos> <sos> KING", max_words=50)
    print(sampled_text)


Generations from Trigram model
<sos> <sos> KING LEWIS understanding quickly lordship Stands Stands ELBOW summers lovest pray Antigonus whistle sour alliance Welshmen Where changed ho far butchers marry Remember caitiff defy for secure act heavier live Cold manage Troy Faith wait what side changes LEWIS pays felt OVERDONE groan Bound torments between obedient chosen herein innocents While
<sos> <sos> KING please salt vast thou'rt loose try I. absolute Clarence complexions division bone sons dangers breaths infant Richmond shop Gentle worldly sad accused 'em sits powers pursuit ruthless Gardener dash Musician sued mightst conquer Exeter choler complexion put move the gods ; pines riches king rushes griefs hills fairest BLUNT March
<sos> <sos> KING EDWARD IV : So thus befall empty respected arrived Go bury Thank unhappy sickness moon sway shot Why thinking imperial shut thousands coldly Masters arms fair slave subtle 'not health escape apace Nor corruption vice companion her poor deceit w

In [130]:
np.random.seed(42)
random.seed(42)
print("Generations from 4-gram model with Add-k smoothing")
fourgram_lm = WordNGramLMWithAddKSmoothing(4, k=0.01)
fourgram_lm.fit(train_data)
for _ in range(20):
    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

Generations from 4-gram model with Add-k smoothing
<sos> <sos> <sos> KING LEWIS unfold quickly lordship Stands Stands ELBOW summers lovest pray Antigonus whistle sour alliance Welshmen Where changed ho far butchers marry Remember caitiff defy for secure act heavier live Cold manage Troy Faith wait what side changes LEWIS pays felt OVERDONE groan Bound torments between obedient chosen herein innocents While
<sos> <sos> <sos> KING please salt vast thou'rt loose try I. absolute Clarence complexions division bone sons dangers breaths infant Richmond shop Gentle worldly sad accused 'em sits powers pursuit ruthless Gardener dash Musician sued mightst conquer Exeter choler complexion put move the gall None prevented riches king rushes griefs hills fairest BLUNT March
<sos> <sos> <sos> KING EDWARD IV : So tidings befall empty respected arrived Go bury Thank unhappy sickness moon sway shot Why thinking imperial shut thousands coldly Masters arms fair slave subtle 'not health escape apace Nor co

In [131]:
np.random.seed(42)
random.seed(42)
print("Generations from 5-gram model with Add-k smoothing")
fivegram_lm = WordNGramLMWithAddKSmoothing(5, k=0.01)
fivegram_lm.fit(train_data)
for _ in range(20):
    sampled_text = fivegram_lm.sample_text("<sos> <sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

Generations from 5-gram model with Add-k smoothing
<sos> <sos> <sos> <sos> KING LEWIS unfold quickly lordship Stands Stands ELBOW summers lovest pray Antigonus whistle sour alliance Welshmen Where changed ho far butchers marry Remember caitiff defy for secure act heavier live Cold manage Troy Faith wait what side changes LEWIS pays felt OVERDONE groan Bound torments between obedient chosen herein innocents While
<sos> <sos> <sos> <sos> KING please salt vast thou'rt loose try I. absolute Clarence complexions division bone sons dangers breaths infant Richmond shop Gentle worldly sad accused 'em sits powers pursuit ruthless Gardener dash Musician sued mightst conquer Exeter choler complexion put move the gall None prevented riches king rushes griefs hills fairest BLUNT March
<sos> <sos> <sos> <sos> KING EDWARD IV : So tidings befall empty respected arrived Go bury Thank unhappy sickness moon sway shot Why thinking imperial shut thousands coldly Masters arms fair slave subtle 'not health e

In [51]:
# </NO_AUTOGRADE>

### Write-Up Question 4 (1 Point)

Write expression for unigram, trigram, 4-gram, and 5-gram models with Laplace smoothing. (Your answer to this question should go in your separate write-up PDF.)

### Write-Up Question 5 (3 Points)

Plot how the train and dev perplexities of bigram, trigram, 4-gram, and 5-gram LMs with Add-k smoothing from different values of k -- $\{1e-8, 1e-7, \cdots, 1e-1, 1\}$. You should have perplexity on the y-axis and k on the x axis. Use log-scaling for the x axis when plotting. Explain the trend that you see in 3 lines. Finally, report the best setup i.e. values of $N$ and $k$ which achieve the best dev accuracy. (Your answer to this question should go in your separate write-up PDF; the following code block below is for the computation you'll need to do in preparation for your write-up.)

In [52]:
# <NO_AUTOGRADE>

In [53]:

# YOUR CODE HERE


In [54]:
# </NO_AUTOGRADE>

### Exercise 3.3 Language Model Interpolation (10 Points)

An alternate to smoothing that often works well in practice is interpolating between different language models. Let's say we are trying to compute $P(w_n \mid w_{n-2} w_{n-1})$, but we have no examples of the particular trigram $w_{n-2}, w_{n-1} w_n$ in the training corpus, we can instead estimate its probability by using the bigram probability $P(w_n \mid w_{n-1})$. If there are no examples of the bigram $w_{n-1} w_n$ in the training data either, we use the unigram probability $P(w_n)$. Formally, the trigram probability by mixing the three distributions is given by:

$$\hat{P}(w_n \mid w_{n-2} w_{n-1}) = \lambda_1 P(w_n) + \lambda_2 P(w_n \mid w_{n-1}) + \lambda_3 P(w_n \mid w_{n-1} w_{n-2})$$

where $\lambda_1 + \lambda_2 + \lambda_3 = 1$ (and each $\lambda$ is non-negative), making the above equation a form of weighted averaging. We can similarly write expressions for other N-gram LMs.

But how do we choose the values of different $\lambda_i$? We choose these values by tuning them on a held out data i.e. the dev set, very similar to tuning hyperparameters for a machine learning model.

In this exercise, you will implement the class `WordNGramLMWithInterpolation` similar to `WordNGramLM` and `WordNGramLMWithAddKSmoothing` that you did in the previous exercises but this time to support interpolation between different LMs.

In [160]:
class WordNGramLMWithInterpolation(WordNGramLM):
    """
    Interpolated N-Gram Language Model.
    """

    def __init__(self, N: int, lambdas: List[float]):
        """
        Constructor for WordNGramLMWithInterpolation class.
        Inputs:
            - N: int, the N in N-gram
            - lambdas: List[float], weights for 1-gram, 2-gram, ... N-gram
        """
        super().__init__(N)
        self.lambdas = lambdas
        if len(lambdas) != N:
            raise ValueError("Length of lambdas must match N")

        # Create sub-models
        # These inherit from WordNGramLM, so they assume they need to add <eos>
        self.sub_models = [WordNGramLM(n) for n in range(1, N + 1)]
        self._pred_vocab = []
        self._V = 0

    def _clean_input(self, data: List[str]) -> List[str]:
        """
        Helper: Removes <eos> from the end of input lines if present.
        This prevents 'WordNGramLM.fit' from adding a second <eos>,
        creating the 'word <eos> <eos>' pattern.
        """
        cleaned = []
        for line in data:
            line = line.strip()
            # If the dataset already has <eos> (like train_data_wth_unks), remove it.
            if line.endswith("<eos>"):
                line = line[:-5].strip()
            cleaned.append(line)
        return cleaned

    def fit(self, train_data: List[str]):
        """
        Trains an N-gram language model with interpolation.
        """
        # 1. Clean the data (Strip existing <eos>)
        # The sub-models' fit() method will add one <eos> back automatically.
        clean_data = self._clean_input(train_data)

        # 2. Train sub-models on clean data
        for model in self.sub_models:
            model.fit(clean_data)

        # 3. Share vocabulary from the largest model
        self.vocab = self.sub_models[-1].vocab

        # 4. Prepare prediction vocab
        if self.N > 1:
            self._pred_vocab = sorted(list(self.vocab - {"<sos>"}))
        else:
            self._pred_vocab = sorted(list(self.vocab))

        self._V = len(self._pred_vocab)

    def eval_perplexity(self, eval_data: List[str]) -> float:
        """
        Evaluates the perplexity of the N-gram language model with interpolation.
        """
        # 1. Clean the input (Strip existing <eos>)
        clean_data = self._clean_input(eval_data)

        total_log_prob = 0.0
        total_tokens = 0

        for sent in clean_data:
            # 2. Add EXACTLY ONE <eos> for evaluation
            sent_with_eos = sent + " <eos>"
            raw_tokens = sent_with_eos.split()

            # Map OOV tokens
            tokens = [self._map_oov_to_unk(t) for t in raw_tokens]

            # Pad with <sos> for the highest order N
            k_max = self.N - 1
            padded_tokens = (["<sos>"] * k_max) + tokens

            # Iterate over the sequence (including the one <eos>)
            for i in range(k_max, len(padded_tokens)):
                w = padded_tokens[i]
                if w == "<sos>": continue

                step_prob = 0.0

                # Interpolate probabilities
                for n_idx, model in enumerate(self.sub_models):
                    order = n_idx + 1
                    weight = self.lambdas[n_idx]

                    k_sub = order - 1
                    if k_sub == 0:
                        ctx = tuple()
                    else:
                        ctx = tuple(padded_tokens[i-k_sub : i])

                    # Get MLE count-based probability
                    count_w = model.next_counts[ctx].get(w, 0)
                    count_ctx = model.context_counts.get(ctx, 0)

                    if count_ctx > 0:
                        p_model = count_w / count_ctx
                    else:
                        p_model = 0.0

                    step_prob += weight * p_model

                if step_prob <= 0:
                    return float("inf")

                total_log_prob += math.log(step_prob)
                total_tokens += 1

        if total_tokens == 0:
            return 1.0

        return math.exp(-total_log_prob / total_tokens)

    def sample_text(self, prefix: str = "<sos>", max_words: int = 100) -> str:
        """
        Samples text from the interpolated model.
        """
        if prefix.strip():
            prefix_tokens = prefix.strip().split()
        else:
            prefix_tokens = []

        # Ensure enough SOS tokens
        k_max = self.N - 1
        lead = 0
        while lead < len(prefix_tokens) and prefix_tokens[lead] == "<sos>":
            lead += 1
        prefix_tokens = (["<sos>"] * k_max) + prefix_tokens[lead:]

        prefix_tokens = [t if t == "<sos>" else self._map_oov_to_unk(t) for t in prefix_tokens]
        out = list(prefix_tokens)

        pred_vocab = self._pred_vocab
        V_size = len(pred_vocab)

        for _ in range(max_words):
            probs = np.zeros(V_size)

            for n_idx, model in enumerate(self.sub_models):
                order = n_idx + 1
                weight = self.lambdas[n_idx]

                k_sub = order - 1
                if k_sub == 0:
                    ctx = tuple()
                else:
                    ctx = tuple(out[-k_sub:])

                count_ctx = model.context_counts.get(ctx, 0)

                if count_ctx > 0:
                    # Vectorized probability calculation
                    current_counts = np.array([model.next_counts[ctx].get(w, 0) for w in pred_vocab])
                    current_probs = current_counts / count_ctx
                    probs += weight * current_probs

            prob_sum = probs.sum()
            if prob_sum == 0:
                break
            probs = probs / prob_sum

            nxt = np.random.choice(pred_vocab, p=probs)
            out.append(nxt)
            if nxt == "<eos>":
                break

        return " ".join(out)

    def _map_oov_to_unk(self, tok: str) -> str:
        return tok if tok in self.vocab else "<unk>"

In [56]:
# <NO_AUTOGRADE>

Test implementation of `eval_perplexity` for trigram model with Interpolation

In [161]:
trigram_lm = WordNGramLMWithInterpolation(3, [0.3, 0.3, 0.4])
trigram_lm.fit(train_data_wth_unks)

train_ppl = trigram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = trigram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Trigram model with Interpolation: {train_ppl}")
print(f"Dev Perplexity for Trigram model with Interpolation: {dev_ppl}")

Train Perplexity for Trigram model with Interpolation: 11.293389655622214
Dev Perplexity for Trigram model with Interpolation: 123.68182574518035


You should see a train perplexity of around 13 and dev perplexity of 139. This should also be the best performing model based on dev perplexity that we have seen so far.

Test implementation of `eval_perplexity` for 4-gram model with Interpolation

In [151]:
fourgram_lm = WordNGramLMWithInterpolation(4, [0.2, 0.4, 0.3, 0.1])
fourgram_lm.fit(train_data_wth_unks)

train_ppl = fourgram_lm.eval_perplexity(train_data_wth_unks)
dev_ppl = fourgram_lm.eval_perplexity(dev_data)
print(f"Train Perplexity for Trigram model with Interpolation: {train_ppl}")
print(f"Dev Perplexity for Trigram model with Interpolation: {dev_ppl}")

Train Perplexity for Trigram model with Interpolation: 6.207159821117634
Dev Perplexity for Trigram model with Interpolation: 127.21981232925698


Here you should get a train perplexity of roughly 8 and dev perplexity around 144.

Play around with different values of $\lambda_i$ and see how it effects the train and dev perplexities.

In [152]:
np.random.seed(42)
random.seed(42)

print("Generations from Trigram model with Interpolation")
trigram_lm = WordNGramLMWithInterpolation(3, [0.3, 0.3, 0.4])
trigram_lm.fit(train_data)
for _ in range(20):
    sampled_text = trigram_lm.sample_text("<sos> <sos> KING", max_words=50)
    print(sampled_text)

Generations from Trigram model with Interpolation
<sos> <sos> KING HENRY steps in foreign Romeo , : poison from so , with the Bolingbroke , And bear some other York and Edward , and he should KING RICHARD III : Stanley , <eos>
<sos> <sos> KING soul usurer her Her <eos>
<sos> <sos> KING RICHARD III : Noble . since He did : O Paulina , what serpent that will my wife 's <unk> , Since fate , my good can not <unk> self , word ill Edward ! Once more of thine , Thou <unk> the mind ; <eos>
<sos> <sos> KING HENRY VI KING LEWIS him again . <eos>
<sos> <sos> KING RICHARD III <unk> balm of my , <eos>
<sos> <sos> KING . <unk> <unk> and to be determined he carries <unk> <unk> , vent hence for the right , soft ; one tyranny But , O , that to 's is not Oxford , Vouchsafe to his grave in my wife to The gates are gentlemen ! <eos>
<sos> <sos> KING RICHARD : Brother of Gloucester 's death with <unk> my sense . <eos>
<sos> <sos> KING HENRY VI Froth Marcius , then ; Ay , in sight ! <eos>
<sos> <sos> KING E

In [153]:
np.random.seed(42)
random.seed(42)

print("Generations from 4-gram model with Interpolation")
fourgram_lm = WordNGramLMWithInterpolation(4, [0.2, 0.4, 0.3, 0.1])
fourgram_lm.fit(train_data)
for _ in range(20):
    sampled_text = fourgram_lm.sample_text("<sos> <sos> <sos> KING", max_words=50)
    print(sampled_text)

Generations from 4-gram model with Interpolation
<sos> <sos> <sos> KING HENRY of such but <unk> , <unk> their hearts from -- why not FRIAR LAURENCE : His lordship he attain his <unk> <unk> <unk> Upon the Tower . <eos>
<sos> <sos> <sos> KING <unk> here , <unk> war was now O , now I <unk> her , those cold ways , and lives . the red with their helps only <eos>
<sos> <sos> <sos> KING EDWARD : I do bend their bows ; I , that : you out ; 'd the noble if thou : <unk> , the like a <unk> <eos>
<sos> <sos> <sos> KING HENRY VI A the friend , how shall poor humble swain , NORTHUMBERLAND , <unk> 'd man , and to be father had been <eos>
<sos> <sos> <sos> KING HENRY BOLINGBROKE living low I thy throat , thou keep'st the stroke Betwixt dread <unk> , she that 's like me . <eos>
<sos> <sos> <sos> KING HENRY most assured That ne'er did violence spare enmity . <eos>
<sos> <sos> <sos> KING HENRY <eos>
<sos> <sos> <sos> KING RICHARD III : Enough then Aufidius , and you ? <eos>
<sos> <sos> <sos> KING RICHARD

### Write-Up Question 6 (3 Points)

For bigram, trigram, and 4-gram models with interpolation, train models with different values of $\lambda_i$ and evaluate perplexities on train and dev datasets. You should try at least 5 sets of values for each model. Report the $\lambda_i$ values that you experiment with and train and dev perplexities for each setting and N-gram model. (Your answer to this question should go in your separate write-up PDF; the following code block below is for the computation you'll need to do in preparation for your write-up.)

In [ ]:

# YOUR CODE HERE


In [ ]:
# </NO_AUTOGRADE>